<a href="https://colab.research.google.com/github/lucguittard/DS-Unit-1-Sprint-3-Statistical-Tests-and-Experiments/blob/master/Luc_Guittard_LS_DS_131_Statistics_Probability_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

## *Data Science Unit 1 Sprint 3 Assignment 1*

# Apply the t-test to real data

Your assignment is to determine which issues have "statistically significant" differences between political parties in this [1980s congressional voting data](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records). The data consists of 435 instances (one for each congressperson), a class (democrat or republican), and 16 binary attributes (yes or no for voting for or against certain issues). Be aware - there are missing values!

Your goals:

1. Load and clean the data (or determine the best method to drop observations when running tests)
2. Using hypothesis testing, find an issue that democrats support more than republicans with p < 0.01
3. Using hypothesis testing, find an issue that republicans support more than democrats with p < 0.01
4. Using hypothesis testing, find an issue where the difference between republicans and democrats has p > 0.1 (i.e. there may not be much of a difference)

Note that this data will involve *2 sample* t-tests, because you're comparing averages across two groups (republicans and democrats) rather than a single group against a null hypothesis.

Stretch goals:

1. Refactor your code into functions so it's easy to rerun with arbitrary variables
2. Apply hypothesis testing to your personal project data (for the purposes of this notebook you can type a summary of the hypothesis you formed and tested)

In [1]:
### YOUR CODE STARTS HERE

!wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data

--2019-08-19 18:57:41--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.03s   

2019-08-19 18:57:41 (594 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



In [4]:
import pandas as pd
import numpy as np

df = pd.read_csv('house-votes-84.data', 
                 header=None,
                 names=['party','handicapped-infants','water-project',
                          'budget','physician-fee-freeze', 'el-salvador-aid',
                          'religious-groups','anti-satellite-ban',
                          'aid-to-contras','mx-missile','immigration',
                          'synthetic-fuels', 'education', 'right-to-sue','crime','duty-free',
                          'south-africa'])
print(df.shape)
df.head()

(435, 17)


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synthetic-fuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [29]:
df.isna().sum() #no nulls or nans being reported by functions isna() and isnull(), just question marks that need replacing

df2 = df.replace({'?': np.nan, 'n':0, 'y':1})
df2.head()

print(df2.isna().sum()) #..oh
#df2.isnull().sum() - df2.isna().sum() == 0

party                     0
handicapped-infants      12
water-project            48
budget                   11
physician-fee-freeze     11
el-salvador-aid          15
religious-groups         11
anti-satellite-ban       14
aid-to-contras           15
mx-missile               22
immigration               7
synthetic-fuels          21
education                31
right-to-sue             25
crime                    17
duty-free                28
south-africa            104
dtype: int64


In [32]:
df3 = df2.fillna(0) # no-vote equated to non-support for issue in question
df3.head()
#df3.isna().sum()

,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synthetic-fuels,education,right-to-sue,crime,duty-free,south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0
2,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0


In [36]:
#inferential stats time
from scipy.stats import ttest_ind #assuming independence between parties

#separating into two dfs data for each party
rep_df = df3[df3['party'] == 'republican']
dem_df = df3[df3['party'] == 'democrat']
dem_df.head()


,party,handicapped-infants,water-project,budget,physician-fee-freeze,el-salvador-aid,religious-groups,anti-satellite-ban,aid-to-contras,mx-missile,immigration,synthetic-fuels,education,right-to-sue,crime,duty-free,south-africa
2,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0
5,democrat,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
6,democrat,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0


In [37]:
#running the t-test(s)
ttest_ind(rep_df['handicapped-infants'],dem_df['handicapped-infants'])

# null-h = no difference in voting trend b/w parties on examined issue
# p-value = 1.57e-17 --> much less than 0.05 - null is rejected, there is a 
# high confidence in the alt. hypothesis: dems and reps vote differently on this issue


Ttest_indResult(statistic=-8.897130738692912, pvalue=1.5743382054891396e-17)

In [41]:
print(rep_df['handicapped-infants'].sum() - dem_df['handicapped-infants'].sum()) 

# dems favored support on this issue much more so than reps

-125.0


In [59]:
#now lets cycle through the other policy issues, looking for issues where rep support significantly greater than dem support
def support(column):
  return column.sum() 

rep_support = rep_df.apply(support)
dem_support = dem_df.apply(support) #156 - 31 = 125 

rep_support


party                   republicanrepublicanrepublicanrepublicanrepubl...
handicapped-infants                                                    31
water-project                                                          75
budget                                                                 22
physician-fee-freeze                                                  163
el-salvador-aid                                                       157
religious-groups                                                      149
anti-satellite-ban                                                     39
aid-to-contras                                                         24
mx-missile                                                             19
immigration                                                            92
synthetic-fuels                                                        21
education                                                             135
right-to-sue                          

In [60]:
dem_support

party                   democratdemocratdemocratdemocratdemocratdemocr...
handicapped-infants                                                   156
water-project                                                         120
budget                                                                231
physician-fee-freeze                                                   14
el-salvador-aid                                                        55
religious-groups                                                      123
anti-satellite-ban                                                    200
aid-to-contras                                                        218
mx-missile                                                            188
immigration                                                           124
synthetic-fuels                                                       129
education                                                              36
right-to-sue                          

In [61]:
#budget, of course, is a highly partisan policy issue, with high republican support here
ttest_ind(rep_df['budget'],dem_df['budget'])

#yeah, with a p-value of approx. 5e-72, there is little doubt that the null may be rejected; 
# there is a difference in voting preferences of members of the two parties on the matter of budget.

Ttest_indResult(statistic=-21.88266384579222, pvalue=4.994834925373016e-72)

In [66]:
#where is there (relatively) little partisan contention? Umm, what about that issue that recalls the 1st amendment..

ttest_ind(rep_df['religious-groups'],dem_df['religious-groups']) 

#t measure indicates difference between the means of sampled distributions

Ttest_indResult(statistic=9.874683331966077, pvalue=7.088016917557037e-21)

In [67]:
# surprising result of 7.09e-21 p-value (above); what does this mean? big t and small p ->  null can be rejected; there is actually a difference 
# in the vote across party-lines for this issue

ttest_ind(rep_df['water-project'],dem_df['water-project']) 

Ttest_indResult(statistic=-0.06131212998472336, pvalue=0.9511389201775375)

In [0]:
#contrary to the inferential stats of the previous policy issue, here they point to bipartisan commonality; 
# a very high p-value of 0.95 indicates that the null-hypothesis cannot/should not be rejected. 

In [0]:
#stretching out here

#part 1, defining functions and looping

# from google.colab import files
# files.upload()

def ttest (column):
  return ttest_ind(rep_df[column],dem_df[column])

#ttest('budget') , only works for predefined dfs ; try making a loop for df columns

def tttest (df):  #for loop
  for column in df:
    return ttest_ind(df[column],df[column]) #no way is this gonna work; need to cycle through the columns in a defined manner no?
                                            
tttest(rep_df) # nope 


#part2, apply to your project -> will do

# might need the following: 
# from scipy.stats import ttest_1samp
# from scipy.stats import ttest_ind
# ..

